# Quasi-Monte Carlo Generators, Randomizers, and Fast Transforms

## Setup

In [1]:
import qmcpy as qp
import numpy as np
import scipy.stats
import timeit
from matplotlib import pyplot
from collections import OrderedDict
import os
import time
import itertools

In [2]:
pyplot.style.use("seaborn-v0_8-pastel")
colors = [
    "xkcd:purple",
    "xkcd:green",
    "xkcd:blue",
    "xkcd:pink",
    "xkcd:brown",
    "xkcd:red",
    "xkcd:orange",
    "xkcd:violet",
    "xkcd:grey",
]

## Snippets

In [16]:
%%time 
lattice = qp.Lattice(
    dimension = 52,
    randomize = "shift", # for unrandomized lattice set randomize = None
    replications = 16, # R
    order = "radical inverse", # also supports "linear" and "Gray code"
    seed = None, # pass integer seed for reproducibility
    generating_vector = "LDData:mps.exod2_base2_m20_CKN.txt")
x = lattice(2**16) # a numpy.ndarray with shape 16 x 65536 x 52

CPU times: user 1.15 s, sys: 142 ms, total: 1.29 s
Wall time: 1.46 s


In [ ]:
%%time
dnb2 = qp.DigitalNetB2(
    dimension = 52, 
    randomize = "LMS DS", # Matousek's LMS then a digital shift
    # other options ["NUS", "DS", "LMS", None]
    t = 64, # number of LMS bits i.e. number of rows in S_j
    alpha = 2, # interlacing factor for higher order digital nets
    replications = 16, # R
    order = "radical inverse", # also supports "Gray code"
    seed = None, # pass integer seed for reproducibility
    generating_matrices = "LDData:mps.sobol_Cs.txt")
x = dnb2(2**16) # a numpy.ndarray with shape 16 x 65536 x 52

CPU times: user 1.17 s, sys: 437 ms, total: 1.61 s
Wall time: 1.82 s


In [ ]:
%%time 
halton = qp.Halton(
    dimension = 52, 
    randomize = "LMS PERM", # Matousek's LMS then a digital permutation
    # other options ["LMS DS", "LMS", "PERM", "DS", "NUS", "QRNG", None]
    t = 64, # number of LMS digits i.e. number of rows in S_j
    replications = 16, # R
    seed = None) # pass integer seed for reproducibility
x = halton(2**10) # a numpy.ndarray with shape 16 x 1024 x 52

CPU times: user 1.46 s, sys: 336 ms, total: 1.79 s
Wall time: 1.81 s


In [ ]:
assert False

## Pointsets

In [ ]:
m = 13 # n = 2^m
n = 2**m # number of points
d = 2 # dimensions

In [ ]:
pointsets = OrderedDict({
    "IID": qp.IIDStdUniform(d).gen_samples(n),
    "Lattice + Shift": qp.Lattice(d).gen_samples(n),
    r"DN${}_{2}$ + LMS + DS": qp.DigitalNetB2(d,randomize="LMS_DS").gen_samples(n),
    r"DN${}_{2}$ + NUS": qp.DigitalNetB2(d,randomize="NUS").gen_samples(n),
    "Halton + LMS + DP": qp.Halton(d,randomize="LMS_PERM").gen_samples(n),
    "Halton + NUS": qp.Halton(d,randomize="NUS").gen_samples(n),
    r"HO${}_2$ DN${}_{2}$ + LMS + DS": qp.DigitalNetB2(d,randomize="LMS_DS",alpha=2).gen_samples(n),
    r"HO${}_2$ DN${}_{2}$ + NUS": qp.DigitalNetB2(d,randomize="NUS",alpha=2).gen_samples(n),
    r"HO${}_3$ DN${}_{2}$ + LMS + DS": qp.DigitalNetB2(d,randomize="LMS_DS",alpha=3).gen_samples(n),
    r"HO${}_3$ DN${}_{2}$ + NUS": qp.DigitalNetB2(d,randomize="NUS",alpha=3).gen_samples(n),
    r"HO${}_4$ DN${}_{2}$ + LMS + DS": qp.DigitalNetB2(d,randomize="LMS_DS",alpha=4).gen_samples(n),
    r"HO${}_4$ DN${}_{2}$ + NUS": qp.DigitalNetB2(d,randomize="NUS",alpha=4).gen_samples(n)})

In [ ]:
nrows,ncols = 2,6
assert len(pointsets)==(nrows*ncols)
fig,ax = pyplot.subplots(nrows=nrows,ncols=ncols,figsize=(ncols*3,nrows*3))
color = "black"
s = .05
for i,(name,x) in enumerate(pointsets.items()):
    ri,ci = i//ncols,i%ncols
    ax[ri,ci].set_title(name)
    ax[ri,ci].scatter(x[:,0],x[:,1],color=color,s=s)
    ax[ri,ci].set_xlim([0,1]); ax[ri,ci].set_xticks([0,1])
    ax[ri,ci].set_ylim([0,1]); ax[ri,ci].set_yticks([0,1])
    ax[ri,ci].set_aspect(1)
fig.savefig("outputs/pointsets.eps",bbox_inches="tight",format="eps")

## Generation time

In [ ]:
r = 1
d = 1
reps = 11
m_max = 20

In [ ]:
def time_block(pointsets_fns,names):
    p = len(pointsets_fns)
    t = np.nan*np.empty((p,m_max+1),dtype=np.float64)
    times = np.empty(reps,dtype=np.float64)
    for i in range(p):
        print("%25s: "%names[i],end="",flush=True)
        generator = pointsets_fns[i]
        for m in range(0,m_max+1):
            print("%d, "%m,end='',flush=True)
            for r in range(reps):
                t0 = time.process_time()
                x = generator(2**m)
                times[r] = time.process_time()-t0
            t[i,m] = np.mean(times)
            if t[i,m]>=1: break
        print()
    return t 

In [ ]:
pointsets_noho_fns = [
    lambda n: qp.IIDStdUniform(d,replications=r).gen_samples(n),
    lambda n: qp.Lattice(d,replications=r).gen_samples(n),
    lambda n: qp.DigitalNetB2(d,randomize="LMS_DS",replications=r).gen_samples(n),
    lambda n: qp.DigitalNetB2(d,randomize="NUS",replications=r).gen_samples(n),
    lambda n: qp.Halton(d,randomize="LMS_DS",replications=r).gen_samples(n),
    lambda n: qp.Halton(d,randomize="NUS",replications=r).gen_samples(n),]
names_noho = [
    "IID",
    "Lattice + Shift",
    r"DN${}_{2}$ + LMS + DS",
    r"DN${}_{2}$ + NUS",
    "Halton + LMS + DP",
    "Halton + NUS",]
t_noho = time_block(pointsets_noho_fns,names_noho)

In [ ]:
pointsets_dnb2_lms_ds_ho_fns = [
    lambda n: qp.DigitalNetB2(d,randomize="LMS_DS",replications=r,alpha=2).gen_samples(n),
    lambda n: qp.DigitalNetB2(d,randomize="LMS_DS",replications=r,alpha=3).gen_samples(n),
    lambda n: qp.DigitalNetB2(d,randomize="LMS_DS",replications=r,alpha=4).gen_samples(n),]
names_dnb2_lms_ds_ho_fns = [
    r"$\alpha = %d$"%2,
    r"$\alpha = %d$"%3,
    r"$\alpha = %d$"%4,]
t_dnb2_lms_ds_ho = time_block(pointsets_dnb2_lms_ds_ho_fns,names_dnb2_lms_ds_ho_fns)

In [ ]:
pointsets_dnb2_nus_ho_fns = [
    lambda n: qp.DigitalNetB2(d,randomize="NUS",replications=r,alpha=2).gen_samples(n),
    lambda n: qp.DigitalNetB2(d,randomize="NUS",replications=r,alpha=3).gen_samples(n),
    lambda n: qp.DigitalNetB2(d,randomize="NUS",replications=r,alpha=4).gen_samples(n),]
names_dnb2_nus_ho = [
    r"$\alpha = %d$"%2,
    r"$\alpha = %d$"%3,
    r"$\alpha = %d$"%4,]
t_dnb2_nus_ho = time_block(pointsets_dnb2_nus_ho_fns,names_dnb2_nus_ho)

In [ ]:
x_fft  = np.random.rand(2**(m_max+1))+1j*np.random.rand(2**(m_max+1))
x_fwht = np.random.rand(2**(m_max+1))
ft_fns = [
    lambda n: qp.fftbr(x_fft[:n]),
    lambda n: qp.ifftbr(x_fft[:n]),
    lambda n: qp.fwht(x_fwht[:n]),]
names_ft = [
    "FFT BR",
    "IFFT BR",
    "FWHT",]
t_ft = time_block(ft_fns,names_ft)

In [ ]:
fig,ax = pyplot.subplots(nrows=1,ncols=3,figsize=(4*5,1*5),sharey=True,sharex=True)
mvec = np.arange(2,m_max+1)
nvec = 2**mvec
linewidth = 3
ms = 7.5
ax[0].plot(nvec,t_noho[0,mvec],label=names_noho[0],marker="o",markersize=ms,linewidth=linewidth,color=colors[0])
ax[0].plot(nvec,t_noho[1,mvec],label=names_noho[1],marker="x",markersize=ms,linewidth=linewidth,color=colors[1])
ax[0].plot(nvec,t_noho[2,mvec],label=names_noho[2],marker="D",markersize=ms,linewidth=linewidth,color=colors[2])
ax[0].plot(nvec,t_noho[3,mvec],label=names_noho[3],marker="s",markersize=ms,linewidth=linewidth,color=colors[3])
ax[0].plot(nvec,t_noho[4,mvec],label=names_noho[4],marker="^",markersize=ms,linewidth=linewidth,color=colors[4])
ax[0].plot(nvec,t_noho[5,mvec],label=names_noho[5],marker="*",markersize=ms,linewidth=linewidth,color=colors[5])
ax[1].plot(nvec,t_noho[2,mvec],label=r"$\alpha = 1$",linewidth=linewidth,color=colors[2],marker="D",markersize=ms)
ax[1].plot(nvec,t_noho[3,mvec],label=r"$\alpha = 1$",linewidth=linewidth,color=colors[3],marker="s",markersize=ms,linestyle=(0,(1,1)))
ax[1].plot(nvec,t_dnb2_lms_ds_ho[0,mvec],label=names_dnb2_lms_ds_ho_fns[0],linewidth=linewidth,color=colors[2],marker="o",markersize=ms)
ax[1].plot(nvec,t_dnb2_nus_ho[0,mvec],label=names_dnb2_nus_ho[0],linewidth=linewidth,color=colors[3],marker="o",markersize=ms,linestyle=(0,(1,1)))
ax[1].plot(nvec,t_dnb2_lms_ds_ho[1,mvec],label=names_dnb2_lms_ds_ho_fns[1],linewidth=linewidth,color=colors[2],marker="x",markersize=ms)
ax[1].plot(nvec,t_dnb2_nus_ho[1,mvec],label=names_dnb2_nus_ho[1],linewidth=linewidth,color=colors[3],marker="x",markersize=ms,linestyle=(0,(1,1)))
ax[1].plot(nvec,t_dnb2_lms_ds_ho[2,mvec],label=names_dnb2_lms_ds_ho_fns[2],linewidth=linewidth,color=colors[2],marker="^",markersize=ms)
ax[1].plot(nvec,t_dnb2_nus_ho[2,mvec],label=names_dnb2_nus_ho[2],linewidth=linewidth,color=colors[3],marker="^",markersize=ms,linestyle=(0,(1,1)))
ax[2].plot(nvec,t_ft[0,mvec],label=names_ft[0],linewidth=linewidth,color=colors[6],marker="x",markersize=ms)
ax[2].plot(nvec,t_ft[1,mvec],label=names_ft[1],linewidth=linewidth,color=colors[7],marker="o",markersize=ms)
ax[2].plot(nvec,t_ft[2,mvec],label=names_ft[2],linewidth=linewidth,color=colors[8],marker="^",markersize=ms)
ax[0].set_title("popular pointsets",fontsize="xx-large")
ax[1].set_title(r"HO${}_{\alpha}$ DN${}_{2}$ + LMS + DS (blue)"+"\n"+r"HO${}_{\alpha}$ DN${}_{2}$ + NUS          (pink)",fontsize="xx-large")
ax[2].set_title("fast transforms",fontsize="xx-large")
ax[0].legend(loc="lower center",frameon=False,bbox_to_anchor=(.5,-.4),ncol=2,fontsize="large")
ax[1].legend(loc="lower center",frameon=False,bbox_to_anchor=(.5,-.35),ncol=4,fontsize="large")
ax[2].legend(loc="lower center",frameon=False,bbox_to_anchor=(.5,-.25),ncol=3,fontsize="large")
ax[0].set_xscale('log',base=2)
for i in range(3):
    ax[i].set_xlabel(r'$n$',fontsize="xx-large")
    ax[i].set_ylabel('time (sec)',fontsize="xx-large")
#ax[0].set_xlim([nvec.min(),nvec.max()])
#ax[0].set_xticks(nvec)
ax[0].set_yscale('log',base=10)
for i in range(3):
    ax[i].yaxis.set_tick_params(labelleft=True)
fig.savefig("outputs/timing.eps",bbox_inches="tight",format="eps")

## Convergence

In [ ]:
r = 300
m_max = 17
seed = 7

In [ ]:
def convergence_block(integrand, transforms, pointsets_fns, names):
    d = integrand.true_measure.discrete_distrib.d
    p = len(pointsets_fns)
    t = np.nan*np.empty((p,m_max+1),dtype=np.float64)
    muhat = np.nan*np.empty((p,m_max+1),dtype=np.float64) 
    rmse = np.nan*np.empty((p,m_max+1),dtype=np.float64)
    for i in range(p):
        print("%25s: "%names[i],end="",flush=True)
        generator = pointsets_fns[i]
        for m in range(0,m_max+1):
            print("%d, "%m,end='',flush=True)
            t0 = timeit.default_timer()
            x = generator(2**m,d)
            t[i,m] = timeit.default_timer()-t0
            y = integrand.f(x)
            muhats = y.mean(1)
            muhat[i,m] = muhats.mean()
            mse = np.mean((muhats-muhat[i,m])**2/(r*(r-1)))
            rmse[i,m] = np.sqrt(mse)
        print()
    return t,muhat,rmse

In [ ]:
# https://www.sfu.ca/~ssurjano/sulf.html
def sulfer_func(t):
    Tr       = t[...,0]
    fAc      = t[...,1]
    fRs      = t[...,2]
    beta_bar = t[...,3]
    Psi_e    = t[...,4]
    f_Psi_e  = t[...,5]
    Q        = t[...,6]
    Y        = t[...,7]
    L        = t[...,8]
    S0 = 1366;
    A  = 5*10**14;
    fact1 = (S0**2) * fAc * (Tr**2) * fRs**2 * beta_bar * Psi_e * f_Psi_e;
    fact2 = 3*Q*Y*L / A;
    DeltaF = -1/2 * fact1 * fact2;
    return DeltaF
sulfer_cf = qp.CustomFun(
    qp.SciPyWrapper(
        qp.IIDStdUniform(9),
        [   scipy.stats.lognorm(scale=0.76, s=np.log(1.2)),
            scipy.stats.lognorm(scale=0.39, s=np.log(1.1)),
            scipy.stats.lognorm(scale=0.85, s=np.log(1.1)),
            scipy.stats.lognorm(scale=0.3,  s=np.log(1.3)),
            scipy.stats.lognorm(scale=5.0,  s=np.log(1.4)),
            scipy.stats.lognorm(scale=1.7,  s=np.log(1.2)),
            scipy.stats.lognorm(scale=71.0, s=np.log(1.15)),
            scipy.stats.lognorm(scale=0.5,  s=np.log(1.5)),
            scipy.stats.lognorm(scale=5.5,  s=np.log(1.5)),]),
    sulfer_func)

In [ ]:
# https://www.sfu.ca/~ssurjano/borehole.html 
def borehole_func(t):
    rw = t[...,0];
    r  = t[...,1];
    Tu = t[...,2];
    Hu = t[...,3];
    Tl = t[...,4];
    Hl = t[...,5];
    L  = t[...,6];
    Kw = t[...,7];
    frac1 = 2 * np.pi * Tu * (Hu-Hl);
    frac2a = 2*L*Tu / (np.log(r/rw)*rw**2*Kw);
    frac2b = Tu / Tl;
    frac2 = np.log(r/rw) * (1+frac2a+frac2b);
    y = frac1 / frac2;
    return y 
borehole_cf = qp.CustomFun(
    qp.SciPyWrapper(
        qp.IIDStdUniform(8),
        [   scipy.stats.norm(loc=0.10,scale=0.0161812),
            scipy.stats.lognorm(scale=np.exp(7.71),s=1.0056),
            scipy.stats.uniform(loc=63070,scale=115600-63070),
            scipy.stats.uniform(loc=990,scale=1110-990),
            scipy.stats.uniform(loc=63.1,scale=116-63.1),
            scipy.stats.uniform(loc=700,scale=820-700),
            scipy.stats.uniform(loc=1120,scale=1680-1120),
            scipy.stats.uniform(loc=9855,scale=12045-9855),]),
    borehole_func)

In [ ]:
# https://www.sfu.ca/~ssurjano/webetal96.html
webster_cf = qp.CustomFun(
    qp.SciPyWrapper(
        qp.IIDStdUniform(2),
        [   scipy.stats.uniform(loc=1,scale=10-1),
            scipy.stats.norm(loc=2,scale=1)]),
    lambda x: x[:,0]**2+x[:,1]**3)

In [ ]:
def oakley_ohagan15_func(t):
    a1 = np.array([0.0118, 0.0456, 0.2297, 0.0393, 0.1177, 0.3865, 0.3897, 0.6061, 0.6159, 0.4005, 1.0741, 1.1474, 0.7880, 1.1242, 1.1982])
    a2 = np.array([0.4341, 0.0887, 0.0512, 0.3233, 0.1489, 1.0360, 0.9892, 0.9672, 0.8977, 0.8083, 1.8426, 2.4712, 2.3946, 2.0045, 2.2621])
    a3 = np.array([0.1044, 0.2057, 0.0774, 0.2730, 0.1253, 0.7526, 0.8570, 1.0331, 0.8388, 0.7970, 2.2145, 2.0382, 2.4004, 2.0541, 1.9845])
    M = np.array([
        [-0.022482886,  -0.18501666,  0.13418263,   0.36867264,   0.17172785,   0.13651143,  -0.44034404, -0.081422854,   0.71321025,  -0.44361072,   0.50383394, -0.024101458, -0.045939684,   0.21666181,  0.055887417],
        [   0.25659630,  0.053792287,  0.25800381,   0.23795905,  -0.59125756, -0.081627077,  -0.28749073,   0.41581639,   0.49752241,  0.083893165,  -0.11056683,  0.033222351,  -0.13979497, -0.031020556,  -0.22318721],
        [ -0.055999811,   0.19542252, 0.095529005,  -0.28626530,  -0.14441303,   0.22369356,   0.14527412,   0.28998481,   0.23105010,  -0.31929879,  -0.29039128,  -0.20956898,   0.43139047,  0.024429152,  0.044904409],
        [   0.66448103,   0.43069872,  0.29924645,  -0.16202441,  -0.31479544,  -0.39026802,   0.17679822,  0.057952663,   0.17230342,   0.13466011,  -0.35275240,   0.25146896, -0.018810529,   0.36482392,  -0.32504618],
        [  -0.12127800,   0.12463327,  0.10656519,  0.046562296,  -0.21678617,   0.19492172, -0.065521126,  0.024404669, -0.096828860,   0.19366196,   0.33354757,   0.31295994, -0.083615456,  -0.25342082,   0.37325717],
        [  -0.28376230,  -0.32820154, -0.10496068,  -0.22073452,  -0.13708154,  -0.14426375,  -0.11503319,   0.22424151, -0.030395022,  -0.51505615,  0.017254978,  0.038957118,   0.36069184,   0.30902452,  0.050030193],
        [ -0.077875893, 0.0037456560,  0.88685604,  -0.26590028, -0.079325357, -0.042734919,  -0.18653782,  -0.35604718,  -0.17497421,  0.088699956,   0.40025886, -0.055979693,   0.13724479,   0.21485613, -0.011265799],
        [ -0.092294730,   0.59209563, 0.031338285, -0.033080861,  -0.24308858, -0.099798547,  0.034460195,  0.095119813,  -0.33801620, 0.0063860024,  -0.61207299,  0.081325416,   0.88683114,   0.14254905,   0.14776204],
        [  -0.13189434,   0.52878496,  0.12652391,  0.045113625,   0.58373514,   0.37291503,   0.11395325,  -0.29479222,  -0.57014085,   0.46291592, -0.094050179,   0.13959097,  -0.38607402,  -0.44897060,  -0.14602419],
        [  0.058107658,  -0.32289338, 0.093139162,  0.072427234,  -0.56919401,   0.52554237,   0.23656926, -0.011782016,  0.071820601,  0.078277291,  -0.13355752,   0.22722721,   0.14369455,  -0.45198935,  -0.55574794],
        [   0.66145875,   0.34633299,  0.14098019,   0.51882591,  -0.28019898,  -0.16032260, -0.068413337,  -0.20428242,  0.069672173,   0.23112577, -0.044368579,  -0.16455425,   0.21620977, 0.0042702105, -0.087399014],
        [   0.31599556, -0.027551859,  0.13434254,   0.13497371,  0.054005680,  -0.17374789,   0.17525393,  0.060258929,  -0.17914162,  -0.31056619,  -0.25358691,  0.025847535,  -0.43006001,  -0.62266361, -0.033996882],
        [  -0.29038151,  0.034101270, 0.034903413,  -0.12121764,  0.026030714,  -0.33546274,  -0.41424111,  0.053248380,  -0.27099455, -0.026251302,   0.41024137,   0.26636349,   0.15582891,  -0.18666254,  0.019895831],
        [  -0.24388652,  -0.44098852, 0.012618825,   0.24945112,  0.071101888,   0.24623792,   0.17484502, 0.0085286769,   0.25147070,  -0.14659862, -0.084625150,   0.36931333,  -0.29955293,   0.11044360,  -0.75690139],
        [  0.041494323,  -0.25980564,  0.46402128,  -0.36112127,  -0.94980789,  -0.16504063, 0.0030943325,  0.052792942,   0.22523648,   0.38390366,   0.45562427,  -0.18631744, 0.0082333995,   0.16670803,   0.16045688]])
    return (a1*t).sum(-1) + (a2*np.sin(t)).sum(-1) + (a3*np.cos(t)).sum(-1) + (t*np.einsum("ij,...j->...i",M,t)).sum(-1)
oakley_ohagan15_cf = qp.CustomFun(
        qp.Gaussian(qp.IIDStdUniform(15)),
        oakley_ohagan15_func)

In [ ]:
def cbeam_func(t):
    R = t[...,0]
    E = t[...,1]
    X = t[...,2]
    Y = t[...,3]
    L = 100;
    D_0 = 2.2535;
    w = 4;
    t = 2;
    Sterm1 = 600*Y / (w*(t**2));
    Sterm2 = 600*X / ((w**2)*t);
    S = Sterm1 + Sterm2;
    Dfact1 = 4*(L**3) / (E*w*t);
    Dfact2 = np.sqrt((Y/(t**2))**2 + (X/(w**2))**2);
    D = Dfact1 * Dfact2
    return D 
cbeam_cf = qp.CustomFun(
    qp.Gaussian(qp.IIDStdUniform(4),mean=[40000,2.9e7,500,1000],covariance=[2000**2,1.45e6**2,100**2,100**2]),
    cbeam_func)

In [ ]:
def G_func(t):
    d = t.shape[-1]
    a = (np.arange(1,d+1)-2)/2
    return ((np.abs(4*t-2)+a)/(1+a)).prod(-1)
G_cf = qp.CustomFun(
    qp.Uniform(qp.IIDStdUniform(3)),
    G_func)

In [ ]:
funcs = [
    qp.CustomFun(
        qp.Uniform(qp.IIDStdUniform(1)),
         lambda x: x[...,0]*np.exp(x[...,0])-1.),
    qp.CustomFun(
        qp.Uniform(qp.IIDStdUniform(2)),
        lambda x: x[...,1]*np.exp(x[...,0]*x[...,1])/(np.exp(1)-2)-1),
    qp.CustomFun(
        qp.Uniform(qp.IIDStdUniform(2),lower_bound=-0.01,upper_bound=0.01),
        lambda x: 5+x[...,0]+x[...,1]+2*np.cos(x[...,0])+2*np.sin(x[...,1])),
    G_cf,
    qp.Genz(qp.IIDStdUniform(3),kind_func='oscillatory',kind_coeff=3),
    qp.Genz(qp.IIDStdUniform(3),kind_func='corner-peak',kind_coeff=2),
    # qp.Ishigami(qp.IIDStdUniform(3)),
    #sulfer_cf,
    #borehole_cf,
    #webster_cf,
    #oakley_ohagan15_cf,
    #cbeam_cf,
]
func_names = [
    r"Simple function, $d=1$",
    r"Simple function, $d=2$",
    r"Oakley & O'Hagan with $d=2$",
    r"G-Function, $d=%d$"%G_cf.d,
    r"Oscillatory Genz, $d=3$",
    r"Corner-peak Genz, $d=3$",
    # "Ishigami",
    # "Sulfer",
    # "Borehole",
    # "Webster",
    #r"Oakley & O'Hagan with $d=15$",
    #"Cantilever Beam",
]
nfuncs = len(funcs)
assert len(func_names)==nfuncs

In [ ]:
pointsets_noho_fns = [
    lambda n,d: qp.IIDStdUniform(d,replications=r,seed=seed).gen_samples(n),
    lambda n,d: qp.Lattice(d,replications=r,seed=seed).gen_samples(n),
    lambda n,d: qp.DigitalNetB2(d,randomize="LMS_DS",replications=r,seed=seed).gen_samples(n),
    lambda n,d: qp.DigitalNetB2(d,randomize="LMS_DS",replications=r,seed=seed,alpha=2).gen_samples(n),
    lambda n,d: qp.DigitalNetB2(d,randomize="LMS_DS",replications=r,seed=seed,alpha=3).gen_samples(n),]
transforms = [
    None,
    "BAKER",
    None,
    None,
    None]
names_noho = [
    "IID",
    "Lattice + Shift",
    r"DN${}_{2}$ + LMS + DS",
    r"HO${}_2$ DN${}_{2}$ + LMS + DS",
    r"HO${}_3$ DN${}_{2}$ + LMS + DS"]
data_noho = [convergence_block(func,transforms,pointsets_noho_fns,names_noho) for func in funcs]

In [ ]:
fig,ax = pyplot.subplots(nrows=2,ncols=3,figsize=(3*5,2*4.5),sharey=False,sharex=True)
ax = np.atleast_1d(ax)
mvec = np.arange(0,m_max+1)
nvec = 2**mvec
linewidth = 3
markers = ["o","x","^","D","*"]
for ii in range(nfuncs):
    i1,i2 = ii//3,ii%3
    t_noho,muhat_noho,rmse_noho = data_noho[ii]
    avgi = rmse_noho[:,mvec[0]].mean()
    for j in range(len(t_noho)):
        ax[i1,i2].plot(nvec,rmse_noho[j,mvec],label=(names_noho[j] if i1==i2==0 else None),linewidth=linewidth,color=colors[j],marker=markers[j])

    # ax[i,1].plot(nvec,rmse_dnb2_lms_ds_ho[0,mvec],label=names_dnb2_lms_ds_ho_fns[0],linewidth=linewidth,color=colors[0])
    # ax[i,1].plot(nvec,rmse_dnb2_lms_ds_ho[1,mvec],label=names_dnb2_lms_ds_ho_fns[1],linewidth=linewidth,color=colors[1])
    # ax[i,1].plot(nvec,rmse_dnb2_lms_ds_ho[2,mvec],label=names_dnb2_lms_ds_ho_fns[2],linewidth=linewidth,color=colors[2])
    # ax[i,1].plot(nvec,rmse_dnb2_lms_ds_ho[3,mvec],label=names_dnb2_lms_ds_ho_fns[3],linewidth=linewidth,color=colors[3])
    # ax[i,1].plot(nvec,rmse_dnb2_lms_ds_ho[4,mvec],label=names_dnb2_lms_ds_ho_fns[4],linewidth=linewidth,color=colors[4])

    # t_dnb2_nus_ho,muhat_dnb2_nus_ho,rmse_dnb2_nus_ho = convergence_block(d,func,pointsets_dnb2_nus_ho_fns,names_dnb2_nus_ho)
    # ax[i,2].plot(nvec,rmse_dnb2_nus_ho[0,mvec],label=names_dnb2_nus_ho[0],linewidth=linewidth,color=colors[0])
    # ax[i,2].plot(nvec,rmse_dnb2_nus_ho[1,mvec],label=names_dnb2_nus_ho[1],linewidth=linewidth,color=colors[1])
    # ax[i,2].plot(nvec,rmse_dnb2_nus_ho[2,mvec],label=names_dnb2_nus_ho[2],linewidth=linewidth,color=colors[2])

    for p,sp in zip([-1/2,-1.,-3/2,-5/2,-7/2],["-1/2","-1","-3/2","-5/2","-7/2"]):
        n0 = 2**mvec[0]
        kappa = avgi/(n0**p)
        nf = 2**mvec[-1]
        lf = kappa*nf**p
        ax[i1,i2].plot([nvec[0],nf],[kappa*n0**p,lf],color="gray",linewidth=1)#,alpha=.5)
        if i2==2:
            ax[i1,i2].text(2*nf,lf,r"$\mathcal{O}(n^{%s})$"%sp,fontsize="xx-large")
    ax[i1,i2].set_yscale('log',base=10)
    ax[i1,i2].set_xlabel(r"$n$",fontsize="xx-large")
    ax[i1,i2].set_title(func_names[ii],fontsize="xx-large");
for i in range(3):
    ax[0,i].xaxis.set_tick_params(labelleft=True)
for i in range(2):
    ax[i,0].set_ylabel("RMSE",fontsize="xx-large")
fig.legend(frameon=False,loc="lower center",bbox_to_anchor=(.5,-.075),ncol=5,fontsize="xx-large")
ax[0,0].set_xscale('log',base=2)
fig.tight_layout()
fig.savefig("outputs/convergence.eps",bbox_inches="tight",format="eps")

## Kernel Methods

In [ ]:
nrows = 2
ncols = 4
m = 8
n = 2**m
fig = pyplot.figure(figsize=(4*ncols,4*nrows),constrained_layout=True)
subfigs = fig.subfigures(nrows=2,ncols=1)
subfigs[0].suptitle(r"shift-invariant $K(u,v) = \tilde{K}((u-v) \;\;\mathrm{mod}\; 1)$",fontsize="xx-large")
subfigs[1].suptitle(r"digitally-shift-invariant $K(u,v) = \tilde{K}(u \oplus v)$",fontsize="xx-large")
ax = np.vstack([subfigs[0].subplots(ncols=4),subfigs[1].subplots(ncols=4)])
xi = np.arange(n) 
xf = xi/n
x1im,x2im = np.meshgrid(xi,xi)
x1i,x2i = x1im.reshape((-1,1)),x2im.reshape((-1,1))
x1m,x2m = x1im/n,x2im/n
x1,x2 = x1m.flatten(),x2m.flatten()
# lattice 
for i,alpha in enumerate([1,2,3]):
    kernel_l = lambda x,z: 1+(2*np.pi*np.ones(2*alpha)/np.arange(1,2*alpha+1)).prod()/((-1)**(alpha+1))*qp.util.bernoulli_poly(2*alpha,(x-z)%1)
    y = kernel_l(xf,xf[0])
    ax[0,0].plot(xf,y,label=r"$\alpha = %d$"%alpha,linewidth=5,color=colors[i])#,alpha=.4)
    ym = kernel_l(x1,x2).reshape((n,n))
    ax[0,i+1].contourf(x1m,x2m,ym,cmap="plasma",levels=100)#,antialiased=True)
    ax[0,i+1].set_title(r"$\alpha = %d$"%alpha)
# digital net
for i,alpha in enumerate([2,3,4]):
    kernel_d = lambda x,z: qp.util.weighted_walsh_funcs(alpha,x^z,t=m)
    y = kernel_d(xi,xi[0])
    y[2**np.arange(m)] = np.nan
    ax[1,0].plot(xf,y,label=r"$\alpha = %d$"%alpha,linewidth=5,color=colors[i])#,alpha=.4)
    ym = kernel_d(x1i,x2i).reshape((n,n))
    ax[1,i+1].contourf(x1m,x2m,ym,cmap="plasma",levels=100)#,antialiased=True)
    ax[1,i+1].set_title(r"$\alpha = %d$"%alpha)
for i in range(nrows):
    ax[i,0].legend(frameon=False,ncol=1,fontsize="large")
    ax[i,0].set_xlabel(r"$\delta$",fontsize="xx-large")
    ax[i,0].set_ylabel(r"$\tilde{K}(\delta)$",fontsize="xx-large")
    for j in range(ncols):
        ax[i,j].set_xlim([0,1]); ax[i,j].set_xticks([0,.25,.5,.75,1])
        if j>=1:
            for spine in ax[i,j].spines.values(): spine.set_visible(False)
            ax[i,j].set_ylim([0,1]); ax[i,j].set_yticks([0,.25,.5,.75,1])
            ax[i,j].set_xlabel(r"$u$",fontsize="xx-large")
            ax[i,j].set_ylabel(r"$v$",fontsize="xx-large")
fig.savefig("outputs/kernels.eps",bbox_inches="tight",format="eps")